In [1]:
%load_ext tensorboard



### Imports

In [2]:
import tensorflow as tf
import os
import pandas as pd
from pprint import pprint
import logging
import multiprocessing
from logging import info, error
from sklearn.preprocessing import MinMaxScaler
from tensorboard.plugins.hparams import api as hp

logging.basicConfig(format='%(asctime)s %(levelname) - 4s %(message)s',
                        level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')

### Global Variables

In [3]:
FEATURE_FIELDS: list = [
        "PRESSURE_REDUCED_TO_MSL_PASCAL",
        "VISIBILITY_AT_SURFACE_METERS",
        "VISIBILITY_AT_CLOUD_TOP_METERS",
        "WIND_GUST_SPEED_AT_SURFACE_METERS_PER_SEC",
        "PRESSURE_AT_SURFACE_PASCAL",
        "TEMPERATURE_AT_SURFACE_KELVIN",
        "DEWPOINT_TEMPERATURE_2_METERS_ABOVE_SURFACE_KELVIN",
        "RELATIVE_HUMIDITY_2_METERS_ABOVE_SURFACE_PERCENT",
        "ALBEDO_PERCENT",
        "TOTAL_CLOUD_COVER_PERCENT"
    ]
LABEL_FIELD: str = "SOIL_TEMPERATURE_0_TO_01_M_BELOW_SURFACE_KELVIN"
USE_CUDA: bool = True

### CUDA ENV exports

In [4]:
def cuda_exports() -> None:
    # Set CUDA and CUPTI paths
    os.environ['CUDA_HOME'] = '/usr/local/cuda'
    os.environ['PATH']= '/usr/local/cuda/bin:$PATH'
    os.environ['CPATH'] = '/usr/local/cuda/include:$CPATH'
    os.environ['LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LIBRARY_PATH'
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/extras/CUPTI/lib64:$LD_LIBRARY_PATH'
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64:$LD_LIBRARY_PATH'

### Loading Pandas DataFrames

In [5]:
def load_data() -> (pd.DataFrame, pd.DataFrame):
    csv_path: str = f"/home/inf0rmatiker/noaa_nam_normalized.csv"
    all_df: pd.DataFrame = pd.read_csv(csv_path, header=0)
    # test_df = all_df.sample(frac=0.2, axis=0)
    # train_df = all_df.drop(index=test_df.index)
    # train_features = train_df[FEATURE_FIELDS]
    # train_labels = train_df[LABEL_FIELD]
    # test_features = test_df[FEATURE_FIELDS]
    # test_labels = test_df[LABEL_FIELD]
    return all_df[FEATURE_FIELDS], all_df[LABEL_FIELD]

### Build and Train Model

In [ ]:
features, labels = load_data()

# Hyperparameter search
batch_sizes = [32, 64, 128, 512]
hl1_num_units = [16, 32, 64]
hl2_num_units = [16, 32, 64]
learning_rates = [0.0001, 0.001, 0.01]

#validation_losses = {}

def session(session_num, batch_size, hl1_num_unit, hl2_num_unit, learning_rate):

    import tensorflow as tf
    import os
    import pandas as pd
    from pprint import pprint
    import logging
    import multiprocessing
    from logging import info, error

    csv_path: str = f"/home/inf0rmatiker/noaa_nam_normalized.csv"
    all_df: pd.DataFrame = pd.read_csv(csv_path, header=0)
    features = all_df[FEATURE_FIELDS]
    labels = all_df[LABEL_FIELD]

    info(f"Running session {session_num}: batch size = {batch_size}, hl1 units = {hl1_num_unit}, hl2 units = {hl2_num_unit}, learning rate = {learning_rate}")
    run_name = f"run_{session_num}_{batch_size}_{hl1_num_unit}_{hl2_num_unit}_{str(learning_rate).replace('.','')}"

    # Stops the model's training if it converges
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",  # Quantity to be monitored.
        min_delta=0.001,  # Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.
        patience=1,  # Number of epochs with no improvement after which training will be stopped.
        verbose=1,  # Verbosity mode, 0 or 1. Mode 0 is silent, and mode 1 displays messages when the callback takes an action.
        mode="min",  # In min mode, training will stop when the quantity monitored has stopped decreasing.
        baseline=0.03,  # Baseline value for the monitored quantity. Training will stop if the model doesn't show improvement over the baseline.
        restore_best_weights=True,  # Whether to restore model weights from the epoch with the best value of the monitored quantity.
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    epochs = 10
    activation = tf.nn.relu

    model = tf.keras.Sequential([
        tf.keras.Input(shape=(len(FEATURE_FIELDS))),
        tf.keras.layers.Dense(units=hl1_num_unit, activation=activation, name="hl_1"),
        tf.keras.layers.Dense(units=hl2_num_unit, activation=activation, name="hl_2"),
        tf.keras.layers.Dense(units=1, name="output_layer")
    ])

    model.compile(loss="mean_squared_error", optimizer=optimizer)
    model.summary()

    # Fit the model to the data
    history = model.fit(
        x=features,
        y=labels,
        shuffle=True,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[
            early_stop_callback
        ]
    )
    hist = pd.DataFrame(history.history)
    info(hist)
    hist.to_csv(f"/home/inf0rmatiker/{run_name}.csv", index=False)
    model.save(f"/home/inf0rmatiker/{run_name}.h5")


session_num = 0
for batch_size in batch_sizes:
    for hl1_num_unit in hl1_num_units:
        for hl2_num_unit in hl2_num_units:
            for learning_rate in learning_rates:

                process_eval = multiprocessing.Process(target=session, args=(session_num, batch_size, hl1_num_unit, hl2_num_unit, learning_rate))
                process_eval.start()
                process_eval.join()
                session_num += 1

1032437/1032437 [==============================] - 1181s 1ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 2/10
1032437/1032437 [==============================] - 1179s 1ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 2: early stopping


24-Apr-22 19:53:58 INFO        loss  val_loss
0  0.001424  0.001251
1  0.001255  0.001228
24-Apr-22 19:54:42 INFO Running session 1: batch size = 32, hl1 units = 16, hl2 units = 16, learning rate = 0.001
2022-04-24 19:54:42.263102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.270459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.270911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.271655: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Ne

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hl_1 (Dense)                (None, 16)                176       
                                                                 
 hl_2 (Dense)                (None, 16)                272       
                                                                 
 output_layer (Dense)        (None, 1)                 17        
                                                                 
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


2022-04-24 19:54:42.646011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.646368: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.646636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 19:54:42.646883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7370 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-04-24 19:54:45.041139: W tensorflow/core/framework/cpu_allocator_impl.

Epoch 1/10
    133/1032437 [..............................] - ETA: 19:51 - loss: 0.0414

...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hl_1 (Dense)                (None, 64)                704       
                                                                 
 hl_2 (Dense)                (None, 32)                2080      
                                                                 
 output_layer (Dense)        (None, 1)                 33        
                                                                 
Total params: 2,817
Trainable params: 2,817
Non-trainable params: 0
_________________________________________________________________


2022-04-25 08:49:26.306611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 08:49:26.306952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 08:49:26.307211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 08:49:26.307450: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6531 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-04-25 08:49:27.798301: W tensorflow/core/framework/cpu_allocator_impl.

Epoch 1/10
    155/1032437 [..............................] - ETA: 17:22 - loss: 0.0086

2022-04-25 08:49:32.042222: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1032437/1032437 [==============================] - 1156s 1ms/step - loss: 9.8688e-04 - val_loss: 0.0010
Epoch 2/10
 336848/1032437 [========>.....................] - ETA: 11:02 - loss: 9.1000e-04